# Libraries

We would use 2 different libraries to practice parallelism.

1. UI processing - `tqdm`  
Library to display and update progress bar.

2. Download processing - `pytube`  
Library to download video from youtube.

### 1. `tqdm` Overview

> Instantly make your loops show a smart progress meter - just wrap any iterable with tqdm(iterable), and you're done!

- link: https://github.com/tqdm/tqdm

In [ ]:
from tqdm import tqdm
from tqdm import trange
from time import sleep

In [ ]:
for i in trange(100):
    sleep(0.01)

In [ ]:
pbar = tqdm(total=100)
for i in range(10):
    sleep(0.1)
    pbar.update(10)
pbar.close()

### 2. `pytube` overview

> `pytube` is a lightweight, Pythonic, dependency-free, library (and command-line utility) for downloading YouTube Videos.

- link: https://pytube.io/en/latest/index.html

In [ ]:
from pytube import YouTube

# YouTube URL to be downloaded
url = "https://www.youtube.com/watch?v=tWFejQSKIYg"
youtube_clip = YouTube(url)

In [ ]:
for item in youtube_clip.streams:
    print(item)

In [ ]:
print(youtube_clip.title)

In [ ]:
print(youtube_clip.thumbnail_url)

In [ ]:
print(youtube_clip.description)

In [ ]:
print(youtube_clip.age_restricted)

- Using callback method from `pytube` library

In [ ]:
def on_complete(stream, file_handle):
    print("Downloaded!")

youtube_clip = YouTube(url,
                       on_complete_callback=on_complete)

In [ ]:
youtube_stream = youtube_clip.streams.filter(progressive=True, file_extension='mp4').first().download("file")